# Quickstart: running JAX on IPU

In [1]:
# Install experimental JAX for IPUs (SDK 3.1) from Github releases.
!pip uninstall -y jax jaxlib
!pip install jax==0.3.16+ipu jaxlib==0.3.15+ipu.sdk310 -f https://graphcore-research.github.io/jax-experimental/wheels.html

Found existing installation: jax 0.3.16+ipu
Uninstalling jax-0.3.16+ipu:
  Successfully uninstalled jax-0.3.16+ipu
Found existing installation: jaxlib 0.3.15+ipu.sdk310
Uninstalling jaxlib-0.3.15+ipu.sdk310:
  Successfully uninstalled jaxlib-0.3.15+ipu.sdk310
Looking in links: https://graphcore-research.github.io/jax-experimental/wheels.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 MB 30.7 MB/s eta 0:00:0000:0100:01


Set the number of IPUs (or use IPUModel emulator).

In [2]:
# Enable IPU compilation cache
import os
os.environ['TF_POPLAR_FLAGS'] = """
  --executable_cache_path=/tmp/ipu-ef-cache
  --show_progress_bar=true
"""

import jax
from jax.config import config

# Uncomment to use IPU model emulator.
# config.FLAGS.jax_ipu_use_model = True
# config.FLAGS.jax_ipu_model_num_tiles = 8

# Select how many IPUs will be visible.
config.update('jax_ipu_device_count', 2)


print(f"Platform={jax.default_backend()}")
print(f"Number of devices={jax.device_count()}")
devices = jax.devices()
print("\n".join([str(d) for d in devices]))

Platform=ipu
Number of devices=2
IpuDevice(id=0, num_tiles=1472, version=ipu2)
IpuDevice(id=1, num_tiles=1472, version=ipu2)


## JAX basics on IPU

Let's run a simple jit function on single IPU

In [3]:
import numpy as np
from jax import jit
import jax.numpy as jnp

@jit
def func(x, w, b):
    return jnp.matmul(w, x) + b

x = np.random.normal(size=[2, 3])
w = np.random.normal(size=[3, 2])
b = np.random.normal(size=[3, 3])

r = func(x, w, b)
print(f"Result:\n{r}")
print(f"Result.platform = {r.platform()}")
print(f"Result.device = {r.device()}")

Result:
[[ 2.2647033   0.03436255  1.272394  ]
 [ 0.85815954 -0.30243903 -0.06740052]
 [-0.87136644  0.67539465 -2.7730992 ]]
Result.platform = ipu
Result.device = IpuDevice(id=0, num_tiles=1472, version=ipu2)


With the `jax.device_put` API, we can put variables onto specific devices. Here is an example to run the jit function on `ipu:1`:

In [4]:
x = jax.device_put(x, devices[1])
w = jax.device_put(w, devices[1])
b = jax.device_put(b, devices[1])

r = func(x, w, b)

print(f"Result:\n{r}")
print(f"Result.platform = {r.platform()}")
print(f"Result.device = {r.device()}")

Result:
[[ 2.2647033   0.03436255  1.272394  ]
 [ 0.85815954 -0.30243903 -0.06740052]
 [-0.87136644  0.67539465 -2.7730992 ]]
Result.platform = ipu
Result.device = IpuDevice(id=1, num_tiles=1472, version=ipu2)


`jit` also allows us to choose the backend the function will be running on.  For example, the below function will be running on `cpu`.

In [5]:
from functools import partial

@partial(jit, backend='cpu')
def func(x, w, b):
    return jnp.matmul(w, x) + b

r = func(x, w, b)

print(f"Result:\n{r}")
print(f"Result.platform = {r.platform()}")
print(f"Result.device = {r.device()}")

Result:
[[ 2.2647033   0.03436255  1.272394  ]
 [ 0.85815954 -0.30243903 -0.06740052]
 [-0.8713663   0.6753946  -2.7730992 ]]
Result.platform = cpu
Result.device = TFRT_CPU_0


## JAX Pseudo Random Numbers generation

Reproducible random numbers across platforms using JAX ThreeFry PRNG.  We will run this on both CPU and IPU.

This is a relatively complex workload for IPU, so the first time through it will take a few seconds to compile.
Let's switch on logging to see the compilation in action.

In [6]:
config.update('jax_log_compiles', True)

In [7]:
def random_fn(seed: int):
    key = jax.random.PRNGKey(seed)
    k1, k2 = jax.random.split(key)
    return k2, jax.random.uniform(k1, (4,))

random_fn_cpu = jax.jit(random_fn, backend="cpu")
print("CPU PRNG:", random_fn_cpu(42))

random_fn_ipu = jax.jit(random_fn, backend="ipu")
print("IPU PRNG:", random_fn_ipu(42))

CPU PRNG: (DeviceArray([255383827, 267815257], dtype=uint32), DeviceArray([0.7367313 , 0.92771065, 0.91349196, 0.3181516 ], dtype=float32))


IPU PRNG: (DeviceArray([255383827, 267815257], dtype=uint32), DeviceArray([0.7367313 , 0.92771065, 0.91349196, 0.3181516 ], dtype=float32))


If you run the same cell again, JAX compilation will be triggered by the redefinition of `random_fn`, but the IPU compilation will hit the cache, and be super quick.

## JAX asynchronous dispatch on IPUs

JAX IPU supports synchronous dispatch, allowing simple and efficient implementation of:
* Inference and training pipeline (see MNIST examples);
* Pipelining between multiple IPUs;

In [8]:
@partial(jit, backend='ipu')
def compute_fn(x, w):
    return jnp.matmul(w, x)

In [ ]:
x = np.random.normal(size=[1024, 1024]).astype(np.float32)
# First run to compile jitted function, and load it on IPU.
compute_fn(x, x)

DeviceArray([[ 14.617263 , -50.7595   ,  20.368622 , ...,  -6.4952183,
               34.855034 , -54.406925 ],
             [ 19.57885  , -12.667716 , -52.3782   , ..., -16.635777 ,
               16.096308 , -27.73622  ],
             [ 25.043592 , -12.324875 ,  24.267931 , ...,  19.963419 ,
                1.868267 ,  32.6991   ],
             ...,
             [-32.77336  ,  -2.1762786,  33.599766 , ..., -69.713974 ,
               16.669662 ,  17.313225 ],
             [ 36.69899  , -12.384048 ,  16.702576 , ..., -40.15704  ,
              -14.347113 ,  12.609639 ],
             [-58.898438 ,   7.7527933,  25.596447 , ..., -64.197975 ,
              -22.680655 ,  11.010284 ]], dtype=float32)

In [ ]:
# No blocking: benchmarking only dispatch time.
%time w = compute_fn(x, x)
w.block_until_ready()

CPU times: user 470 µs, sys: 94 µs, total: 564 µs
Wall time: 410 µs


DeviceArray([[ 14.617263 , -50.7595   ,  20.368622 , ...,  -6.4952183,
               34.855034 , -54.406925 ],
             [ 19.57885  , -12.667716 , -52.3782   , ..., -16.635777 ,
               16.096308 , -27.73622  ],
             [ 25.043592 , -12.324875 ,  24.267931 , ...,  19.963419 ,
                1.868267 ,  32.6991   ],
             ...,
             [-32.77336  ,  -2.1762786,  33.599766 , ..., -69.713974 ,
               16.669662 ,  17.313225 ],
             [ 36.69899  , -12.384048 ,  16.702576 , ..., -40.15704  ,
              -14.347113 ,  12.609639 ],
             [-58.898438 ,   7.7527933,  25.596447 , ..., -64.197975 ,
              -22.680655 ,  11.010284 ]], dtype=float32)

In [ ]:
# Blocking: benchmarking properly the matmul.
%time w = compute_fn(x, x).block_until_ready()

CPU times: user 7.97 ms, sys: 0 ns, total: 7.97 ms
Wall time: 7.18 ms
